In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

raw_data_dir = Path.cwd() / "raw_data"
csv_files = sorted(raw_data_dir.glob("*.csv"))

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {raw_data_dir}")

df_list = []
for p in csv_files:
    tmp = pd.read_csv(p)
    tmp["source_file"] = p.name
    df_list.append(tmp)

df = pd.concat(df_list, ignore_index=True)

df.head()
df.info()

df = df.drop_duplicates(subset=["ASIN"])  # Check for duplicates (same ASIN across files)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   URL                        362 non-null    object 
 1   Image URL                  362 non-null    object 
 2   ASIN                       362 non-null    object 
 3   Title                      362 non-null    object 
 4   Brand                      361 non-null    object 
 5   Fulfillment                362 non-null    object 
 6   Category                   362 non-null    object 
 7   BSR                        362 non-null    int64  
 8   UPC                        144 non-null    float64
 9   GTIN                       68 non-null     float64
 10  EAN                        199 non-null    float64
 11  ISBN                       1 non-null      float64
 12  Subcategory                362 non-null    object 
 13  Subcategory BSR            362 non-null    int64  

In [5]:
### 1.1 Filter category / subcategory (if needed)

# If needed, filter to automotive category and multimeter/analyzer subcategory
auto = df[df["Category"].str.contains("Automotive", case=False, na=False)]

# If Helium10 already pre-filtered, this may be redundant; but you can also:
auto = auto[auto["Subcategory"].str.contains("Multimeter|Analyzer", case=False, na=False)]

### 1.2 Clean numeric columns
num_cols = [
    "Price", "BSR", "ASIN Sales", "ASIN Revenue",
    "Parent Level Sales", "Parent Level Revenue",
    "Review Count", "Reviews Rating",
    "Sales Trend (90 days) (%)", "Price Trend (90 days) (%)",
    "Sales Year Over Year (%)"
]

for c in num_cols:
    if c in auto.columns:
        auto[c] = pd.to_numeric(auto[c], errors="coerce")



In [ ]:
### 2.1 Identify multimeters vs analyzers
auto["is_multimeter"] = auto["Title"].str.contains("multimeter", case=False, na=False)
auto["is_analyzer"]   = auto["Title"].str.contains("analyzer|analyser", case=False, na=False)

### 2.2 Screen / “advanced display” proxies (for later)
screen_keywords = "large screen|large display|big screen|color screen|hd screen|hd display|lcd display|full screen|smart display"
auto["is_large_screen_like"] = auto["Title"].str.contains(screen_keywords, case=False, na=False)

auto["is_rechargeable"] = auto["Title"].str.contains("rechargeable|type-c|usb c|usb-c", case=False, na=False)
auto["is_automotive_targeted"] = auto["Title"].str.contains("car|automotive|vehicle|battery tester", case=False, na=False)



### 2.3 Product type classification
def classify_product(row):
    if row["is_multimeter"] and row["is_analyzer"]:
        return "Multimeter + Analyzer"
    elif row["is_multimeter"]:
        return "Multimeter"
    elif row["is_analyzer"]:
        return "Analyzer"
    else:
        return "Other"

auto["product_type"] = auto.apply(classify_product, axis=1)

### 2.4 Brand cleaning + INNOVA flag
if "Brand" in auto.columns:
    auto["brand_clean"] = (
        auto["Brand"]
        .astype(str)
        .str.strip()
        .str.upper()
    )
else:
    auto["brand_clean"] = "UNKNOWN"

auto["is_innova"] = auto["brand_clean"].str.contains("INNOVA", case=False, na=False)

### 3. Create a “core” dataset for analysis
# Basic sanity filters: drop rows without sales or revenue
core = auto.copy()

if "ASIN Sales" in core.columns:
    core = core[core["ASIN Sales"].notna()]
if "ASIN Revenue" in core.columns:
    core = core[core["ASIN Revenue"].notna()]

# Optional: filter out obviously broken or zero-price listings
if "Price" in core.columns:
    core = core[core["Price"] > 0]

core.info()

### 3.1 Overall market KPIs (last 30 days)
total_revenue = core["ASIN Revenue"].sum()
total_sales   = core["ASIN Sales"].sum()
asin_count    = core["ASIN"].nunique()

avg_price     = core["Price"].mean()
median_price  = core["Price"].median()

avg_rating    = core["Reviews Rating"].mean()
median_rating = core["Reviews Rating"].median()

print("\n=== OVERALL MARKET KPIs (Automotive Multimeters & Analyzers, last 30 days) ===")
print(f"Total revenue (USD): {total_revenue:,.0f}")
print(f"Total units sold:    {total_sales:,.0f}")
print(f"# of unique ASINs:   {asin_count}")
print(f"Average price:       ${avg_price:,.2f} (median ${median_price:,.2f})")
print(f"Average rating:      {avg_rating:.2f} (median {median_rating:.2f})")

### 3.2 INNOVA vs market snapshot
innova_core = core[core["is_innova"]]
other_core  = core[~core["is_innova"]]

def kpi_block(df, label):
    if df.empty:
        print(f"\n[{label}] No data.")
        return
    print(f"\n[{label}]")
    print(f"  Revenue:       ${df['ASIN Revenue'].sum():,.0f}")
    print(f"  Units sold:    {df['ASIN Sales'].sum():,.0f}")
    print(f"  # ASINs:       {df['ASIN'].nunique()}")
    print(f"  Avg price:     ${df['Price'].mean():,.2f}")
    print(f"  Avg rating:    {df['Reviews Rating'].mean():.2f}")
    print(f"  Total reviews: {df['Review Count'].sum():,.0f}")

print("\n=== INNOVA vs OTHER BRANDS ===")
kpi_block(innova_core, "INNOVA")
kpi_block(other_core,  "All other brands")

### 4. Brand-level competitive landscape
brand_summary = (
    core.groupby("brand_clean", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            asin_count=("ASIN", "nunique"),
            avg_price=("Price", "mean"),
            median_price=("Price", "median"),
            avg_rating=("Reviews Rating", "mean"),
            total_reviews=("Review Count", "sum"),
        )
)

# Market share metrics
brand_summary = brand_summary.sort_values("total_revenue", ascending=False)
total_market_rev = brand_summary["total_revenue"].sum()
total_market_sales = brand_summary["total_sales"].sum()

brand_summary["rev_share_%"]  = brand_summary["total_revenue"] / total_market_rev * 100
brand_summary["unit_share_%"] = brand_summary["total_sales"] / total_market_sales * 100

print("\n=== TOP 15 BRANDS BY REVENUE ===")
print(brand_summary.head(15))

### 4.1 Identify top ASINs
top_n = 50
top_asins = (
    core.sort_values("ASIN Revenue", ascending=False)
        .head(top_n)
        .loc[:, [
            "ASIN", "Title", "brand_clean", "product_type",
            "Price", "ASIN Sales", "ASIN Revenue",
            "Review Count", "Reviews Rating",
            "is_large_screen_like", "is_rechargeable", "is_automotive_targeted",
        ]]
)

print("\n=== TOP 20 ASINS BY REVENUE ===")
print(top_asins.head(20))

### 4.2 INNOVA ASINs detail
top_innova_asins = top_asins[top_asins["brand_clean"].str.contains("INNOVA", na=False)]
print("\n=== INNOVA ASINS (IN TOP LIST IF ANY) ===")
print(top_innova_asins)

### 5. Market segmentation by product type
product_type_summary = (
    core.groupby("product_type", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            asin_count=("ASIN", "nunique"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            total_reviews=("Review Count", "sum"),
        )
        .sort_values("total_revenue", ascending=False)
)

product_type_summary["rev_share_%"]  = product_type_summary["total_revenue"] / total_revenue * 100
product_type_summary["unit_share_%"] = product_type_summary["total_sales"] / total_sales * 100

print("\n=== MARKET SPLIT: Multimeter vs Analyzer ===")
print(product_type_summary)

### 6. Price tiering
price_bins   = [0, 30, 60, 100, 200, np.inf]
price_labels = ["<$30", "$30–59", "$60–99", "$100–199", "$200+"]

core["price_tier"] = pd.cut(core["Price"], bins=price_bins, labels=price_labels, right=False)

price_tier_summary = (
    core.groupby("price_tier", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            asin_count=("ASIN", "nunique"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            total_reviews=("Review Count", "sum"),
        )
        .sort_values("total_revenue", ascending=False)
)

price_tier_summary["rev_share_%"]  = price_tier_summary["total_revenue"] / total_revenue * 100
price_tier_summary["unit_share_%"] = price_tier_summary["total_sales"] / total_sales * 100

print("\n=== PRICE TIER SUMMARY ===")
print(price_tier_summary)

### 7. Feature-flag performance (screen / rechargeable / automotive targeting)

feature_flags = ["is_large_screen_like", "is_rechargeable", "is_automotive_targeted"]

feature_summary_list = []

for flag in feature_flags:
    if flag not in core.columns:
        continue
    tmp = (
        core.groupby(flag, as_index=False)
            .agg(
                total_revenue=("ASIN Revenue", "sum"),
                total_sales=("ASIN Sales", "sum"),
                asin_count=("ASIN", "nunique"),
                avg_price=("Price", "mean"),
                avg_rating=("Reviews Rating", "mean"),
                total_reviews=("Review Count", "sum"),
            )
    )
    tmp["feature_flag"] = flag
    feature_summary_list.append(tmp)

feature_summary = pd.concat(feature_summary_list, ignore_index=True) if feature_summary_list else pd.DataFrame()

print("\n=== FEATURE FLAG PERFORMANCE (screen / rechargeable / targeting) ===")
print(feature_summary)

### 7.1 Cross: feature flags within automotive-targeted products only
auto_targeted = core[core["is_automotive_targeted"]]

auto_targeted_feature_summary = []

for flag in feature_flags:
    if flag not in auto_targeted.columns:
        continue
    tmp = (
        auto_targeted.groupby(flag, as_index=False)
            .agg(
                total_revenue=("ASIN Revenue", "sum"),
                total_sales=("ASIN Sales", "sum"),
                asin_count=("ASIN", "nunique"),
                avg_price=("Price", "mean"),
                avg_rating=("Reviews Rating", "mean"),
                total_reviews=("Review Count", "sum"),
            )
    )
    tmp["feature_flag"] = flag
    auto_targeted_feature_summary.append(tmp)

auto_targeted_feature_summary = (
    pd.concat(auto_targeted_feature_summary, ignore_index=True)
    if auto_targeted_feature_summary else pd.DataFrame()
)

print("\n=== FEATURE FLAG PERFORMANCE (within automotive-targeted only) ===")
print(auto_targeted_feature_summary)

### 8. Combine key KPIs for a quick slide-ready table
overall_row = pd.DataFrame([{
    "metric": "Overall automotive DMM market (last 30 days)",
    "total_revenue": total_revenue,
    "total_sales": total_sales,
    "asin_count": asin_count,
    "avg_price": avg_price,
    "median_price": median_price,
    "avg_rating": avg_rating,
    "median_rating": median_rating
}])

innova_row = pd.DataFrame([{
    "metric": "INNOVA",
    "total_revenue": innova_core["ASIN Revenue"].sum(),
    "total_sales": innova_core["ASIN Sales"].sum(),
    "asin_count": innova_core["ASIN"].nunique(),
    "avg_price": innova_core["Price"].mean(),
    "median_price": innova_core["Price"].median(),
    "avg_rating": innova_core["Reviews Rating"].mean(),
    "median_rating": innova_core["Reviews Rating"].median(),
}])

kpi_overview = pd.concat([overall_row, innova_row], ignore_index=True)

print("\n=== HIGH LEVEL KPI OVERVIEW (for slides) ===")
print(kpi_overview)

from pathlib import Path

### 9. Export all key tables to Excel for reporting

# Choose a folder you KNOW is writable on your Mac
output_dir = Path("/Users/sumiaoc/Desktop/DMM_h10/outputs")  # adjust if you like
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "DMM_market_research_summary.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    core.to_excel(writer, sheet_name="auto_filtered_core", index=False)
    brand_summary.to_excel(writer, sheet_name="brand_summary", index=False)
    top_asins.to_excel(writer, sheet_name="top_asins", index=False)
    top_innova_asins.to_excel(writer, sheet_name="innova_asins", index=False)
    product_type_summary.to_excel(writer, sheet_name="product_type", index=False)
    price_tier_summary.to_excel(writer, sheet_name="price_tiers", index=False)
    feature_summary.to_excel(writer, sheet_name="features_all", index=False)
    auto_targeted_feature_summary.to_excel(writer, sheet_name="features_auto_only", index=False)
    kpi_overview.to_excel(writer, sheet_name="kpi_overview", index=False)

print(f"\nExcel summary written to: {output_path}")





<class 'pandas.core.frame.DataFrame'>
Index: 160 entries, 0 to 339
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   URL                        160 non-null    object 
 1   Image URL                  160 non-null    object 
 2   ASIN                       160 non-null    object 
 3   Title                      160 non-null    object 
 4   Brand                      160 non-null    object 
 5   Fulfillment                160 non-null    object 
 6   Category                   160 non-null    object 
 7   BSR                        160 non-null    int64  
 8   UPC                        75 non-null     float64
 9   GTIN                       32 non-null     float64
 10  EAN                        94 non-null     float64
 11  ISBN                       1 non-null      float64
 12  Subcategory                160 non-null    object 
 13  Subcategory BSR            160 non-null    int64  
 14 

/var/folders/5r/n_p5hvzn4ns6rdyy9f5fy4b0q2n_rp/T/ipykernel_74187/1808728429.py:166: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  core.groupby("price_tier", as_index=False)


In [10]:
%pip install streamlit
import streamlit as st



# dmm_dashboard.py

import streamlit as st
import pandas as pd
import numpy as np
import re
from pathlib import Path

# ---------- 1. DATA PREP (based on your existing script) ----------
RAW_DATA_DIR = Path.cwd() / "raw_data"
CSV_PATTERN = "*.csv"
NUM_COLS = [
    "Price",
    "BSR",
    "ASIN Sales",
    "ASIN Revenue",
    "Parent Level Sales",
    "Parent Level Revenue",
    "Review Count",
    "Reviews Rating",
    "Sales Trend (90 days) (%)",
    "Price Trend (90 days) (%)",
    "Sales Year Over Year (%)",
]
REQUIRED_COLS = ["ASIN", "Title", "Category", "Subcategory", "Price", "ASIN Sales", "ASIN Revenue"]


def _list_data_files():
    if not RAW_DATA_DIR.exists():
        return []
    return sorted(RAW_DATA_DIR.glob(CSV_PATTERN))


@st.cache_data
def load_and_prepare_data():
    if not files:
        st.error(f"No CSV files found in {RAW_DATA_DIR} (pattern: {CSV_PATTERN})")
        st.stop()

    df_list = []
    for p in files:
        tmp = pd.read_csv(p)
        tmp["source_file"] = p.name
        df_list.append(tmp)

    df = pd.concat(df_list, ignore_index=True)

    # Drop duplicate ASINs
    df = df.drop_duplicates(subset=["ASIN"])

    # Filter to automotive multimeters/analyzers
    auto = df[df["Category"].astype(str).str.contains("Automotive", case=False, na=False)]
    auto = auto[auto["Subcategory"].astype(str).str.contains("Multimeter|Analyzer", case=False, na=False)]

    # Clean numeric columns
    for c in NUM_COLS:
        if c in auto.columns:
            auto[c] = pd.to_numeric(auto[c], errors="coerce")

    # Flags: multimeter vs analyzer
    title_lower = auto["Title"].fillna("").str.lower()
    auto["is_multimeter"] = title_lower.str.contains("multimeter")
    auto["is_analyzer"] = title_lower.str.contains("analyzer|analyser")

    screen_pattern = re.compile(
        "large screen|large display|big screen|color screen|hd screen|hd display|lcd display|full screen|smart display",
        re.IGNORECASE,
    )
    auto["is_large_screen_like"] = title_lower.str.contains(screen_pattern)

    color_display_pattern = re.compile(
        "color lcd|color screen|full color display|tft display|tft lcd|ips display|oled display",
        re.IGNORECASE,
    )
    auto["is_color_display"] = title_lower.str.contains(color_display_pattern)

    auto["is_rechargeable"] = title_lower.str.contains("rechargeable|type-c|usb c|usb-c")
    auto["is_automotive_targeted"] = title_lower.str.contains("car|automotive|vehicle|battery tester")

    # Product type classification
    auto["product_type"] = np.select(
        [auto["is_multimeter"] & auto["is_analyzer"], auto["is_multimeter"], auto["is_analyzer"]],
        ["Multimeter + Analyzer", "Multimeter", "Analyzer"],
        default="Other",
    )

    # Brand cleaning + INNOVA flag
    if "Brand" in auto.columns:
        auto["brand_clean"] = auto["Brand"].astype(str).str.strip().str.upper()
    else:
        auto["brand_clean"] = "UNKNOWN"

    auto["is_innova"] = auto["brand_clean"].str.contains("INNOVA", case=False, na=False)

    # Core dataset
    core = auto.copy()
    if "ASIN Sales" in core.columns:
        core = core[core["ASIN Sales"].notna()]
    if "ASIN Revenue" in core.columns:
        core = core[core["ASIN Revenue"].notna()]
    if "Price" in core.columns:
        core = core[core["Price"] > 0]

    # Overall KPIs
    total_revenue = core["ASIN Revenue"].sum()
    total_sales = core["ASIN Sales"].sum()
    asin_count = core["ASIN"].nunique()
    avg_price = core["Price"].mean()
    median_price = core["Price"].median()
    avg_rating = core["Reviews Rating"].mean() if "Reviews Rating" in core.columns else np.nan
    median_rating = core["Reviews Rating"].median() if "Reviews Rating" in core.columns else np.nan

    kpi_overview = pd.DataFrame([
        {
            "metric": "Overall market (last 30 days)",
            "total_revenue": total_revenue,
            "total_sales": total_sales,
            "asin_count": asin_count,
            "avg_price": avg_price,
            "median_price": median_price,
            "avg_rating": avg_rating,
            "median_rating": median_rating,
        }
    ])

    # Brand-level summary
    brand_summary = (
        core.groupby("brand_clean", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            asin_count=("ASIN", "nunique"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            total_reviews=("Review Count", "sum"),
        )
        .sort_values("total_revenue", ascending=False)
    )

    # Product type summary
    product_type_summary = (
        core.groupby("product_type", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            asin_count=("ASIN", "nunique"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            total_reviews=("Review Count", "sum"),
        )
        .sort_values("total_revenue", ascending=False)
    )

    # Price tiers
    price_bins = [0, 30, 60, 100, 200, np.inf]
    price_labels = ["<$30", "$30–59", "$60–99", "$100–199", "$200+"]
    core["price_tier"] = pd.cut(core["Price"], bins=price_bins, labels=price_labels, right=False)

    # Large-screen vs non-large-screen price and sales differences
    large_screen_summary = (
        core.groupby("is_large_screen_like", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            asin_count=("ASIN", "nunique"),
        )
    )
    large_screen_summary["segment"] = large_screen_summary["is_large_screen_like"].map(
        {True: "Large-screen-like", False: "Non-large-screen"}
    )

    # Color display vs non-color display premium
    color_display_summary = (
        core.groupby("is_color_display", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            asin_count=("ASIN", "nunique"),
        )
    )
    color_display_summary["segment"] = color_display_summary["is_color_display"].map(
        {True: "Color / advanced display", False: "Non-color display"}
    )

    battery_category_summary = (
        core.groupby("is_rechargeable", as_index=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            asin_count=("ASIN", "nunique"),
        )
    )
    battery_category_summary["segment"] = battery_category_summary["is_rechargeable"].map(
        {True: "Rechargeable", False: "Non-rechargeable / unspecified"}
    )

    price_tier_summary = (
        core.groupby("price_tier", as_index=False, observed=False)
        .agg(
            total_revenue=("ASIN Revenue", "sum"),
            total_sales=("ASIN Sales", "sum"),
            asin_count=("ASIN", "nunique"),
            avg_price=("Price", "mean"),
            avg_rating=("Reviews Rating", "mean"),
            total_reviews=("Review Count", "sum"),
        )
        .sort_values("total_revenue", ascending=False)
    )

    # Top ASINs table
    top_asins = (
        core.sort_values("ASIN Revenue", ascending=False)
        .loc[
            :,
            [
                "ASIN",
                "Title",
                "brand_clean",
                "product_type",
                "Price",
                "ASIN Sales",
                "ASIN Revenue",
                "Review Count",
                "Reviews Rating",
                "is_large_screen_like",
                "is_rechargeable",
                "is_automotive_targeted",
            ],
        ]
    )

    return {
        "core": core,
        "brand_summary": brand_summary,
        "product_type_summary": product_type_summary,
        "price_tier_summary": price_tier_summary,
        "top_asins": top_asins,
        "kpi_overview": kpi_overview,
        "large_screen_summary": large_screen_summary,
        "color_display_summary": color_display_summary,
        "battery_category_summary": battery_category_summary,
    }

# ---------- 2. DASHBOARD LAYOUT ----------

st.set_page_config(
    page_title="Automotive DMM Market Dashboard",
    layout="wide",
)

st.title("Automotive DMMs – Amazon Market Dashboard")
st.caption("Based on Helium10 BlackBox data (last 30 days)")

data = load_and_prepare_data()
core = data["core"]
brand_summary = data["brand_summary"]
product_type_summary = data["product_type_summary"]
price_tier_summary = data["price_tier_summary"]
top_asins = data["top_asins"]
kpi_overview = data["kpi_overview"]
large_screen_summary = data["large_screen_summary"]
color_display_summary = data["color_display_summary"]
battery_category_summary = data["battery_category_summary"]

# ---------- 3. SIDEBAR FILTERS ----------

st.sidebar.header("Filters")

# Brand filter
brands = ["All"] + sorted(brand_summary["brand_clean"].unique().tolist())
selected_brand = st.sidebar.selectbox("Brand", brands, index=0)

# Product type filter
product_types = ["All"] + sorted(core["product_type"].unique().tolist())
selected_ptype = st.sidebar.selectbox("Product type", product_types, index=0)

# Price tier filter
price_tiers = ["All"] + [str(x) for x in price_tier_summary["price_tier"].dropna().astype(str).tolist()]
selected_ptier = st.sidebar.selectbox("Price tier", price_tiers, index=0)

# Feature flags
only_auto_targeted = st.sidebar.checkbox("Only automotive-targeted", value=False)
only_large_screen = st.sidebar.checkbox("Only large-screen-like", value=False)
only_rechargeable = st.sidebar.checkbox("Only rechargeable", value=False)

# ---------- 4. APPLY FILTERS TO CORE ----------

filtered = core.copy()

if selected_brand != "All":
    filtered = filtered[filtered["brand_clean"] == selected_brand]

if selected_ptype != "All":
    filtered = filtered[filtered["product_type"] == selected_ptype]

if selected_ptier != "All":
    # price_tier is a Categorical; cast to string for comparison
    filtered = filtered[filtered["price_tier"].astype(str) == selected_ptier]

if only_auto_targeted:
    filtered = filtered[filtered["is_automotive_targeted"]]

if only_large_screen:
    filtered = filtered[filtered["is_large_screen_like"]]

if only_rechargeable:
    filtered = filtered[filtered["is_rechargeable"]]

# Recompute KPIs for filtered view
if not filtered.empty:
    f_total_revenue = filtered["ASIN Revenue"].sum()
    f_total_sales = filtered["ASIN Sales"].sum()
    f_asin_count = filtered["ASIN"].nunique()
    f_avg_price = filtered["Price"].mean()
    f_avg_rating = filtered["Reviews Rating"].mean() if "Reviews Rating" in filtered.columns else np.nan
else:
    f_total_revenue = f_total_sales = f_asin_count = 0
    f_avg_price = f_avg_rating = np.nan

# ---------- 5. KPI CARDS ----------

st.subheader("Market KPIs – Current View")

col1, col2, col3, col4, col5 = st.columns(5)

col1.metric("Total revenue (USD)", f"${f_total_revenue:,.0f}")
col2.metric("Units sold", f"{f_total_sales:,.0f}")
col3.metric("# of ASINs", f"{f_asin_count}")
col4.metric("Avg price", f"${f_avg_price:,.2f}" if not np.isnan(f_avg_price) else "N/A")
col5.metric("Avg rating", f"{f_avg_rating:.2f}" if not np.isnan(f_avg_rating) else "N/A")

st.markdown("---")
st.subheader("Display feature economics")

col_ls, col_cd, col_batt = st.columns(3)

with col_ls:
    st.caption("Large-screen vs non-large-screen: price and sales comparison (market-wide)")
    st.dataframe(
        large_screen_summary[["segment", "asin_count", "total_sales", "total_revenue", "avg_price", "avg_rating"]],
        hide_index=True,
        use_container_width=True,
    )

with col_cd:
    st.caption("Color vs non-color display: price premium and sales comparison (market-wide)")
    st.dataframe(
        color_display_summary[["segment", "asin_count", "total_sales", "total_revenue", "avg_price", "avg_rating"]],
        hide_index=True,
        use_container_width=True,
    )

with col_batt:
    st.caption("Rechargeable vs non-rechargeable: price and sales comparison (market-wide)")
    st.dataframe(
        battery_category_summary[["segment", "asin_count", "total_sales", "total_revenue", "avg_price", "avg_rating"]],
        hide_index=True,
        use_container_width=True,
    )

# ---------- 6. CHARTS ----------

st.markdown("---")
st.subheader("Brand performance")

# Brand performance (using filtered data)
brand_view = (
    filtered.groupby("brand_clean", as_index=False)
    .agg(
        total_revenue=("ASIN Revenue", "sum"),
        total_sales=("ASIN Sales", "sum"),
        asin_count=("ASIN", "nunique"),
    )
    .sort_values("total_revenue", ascending=False)
    .head(15)
)

colA, colB = st.columns(2)

with colA:
    st.caption("Top brands by revenue (current filters)")
    st.bar_chart(
        brand_view.set_index("brand_clean")["total_revenue"],
        height=350,
    )

with colB:
    st.caption("Top brands by units (current filters)")
    st.bar_chart(
        brand_view.set_index("brand_clean")["total_sales"],
        height=350,
    )

st.markdown("---")
st.subheader("Product type & price tiers")

pt_view = (
    filtered.groupby("product_type", as_index=False)
    .agg(
        total_revenue=("ASIN Revenue", "sum"),
        total_sales=("ASIN Sales", "sum"),
    )
    .sort_values("total_revenue", ascending=False)
)

ptier_view = pd.DataFrame(columns=["price_tier", "total_revenue", "total_sales"])
if "price_tier" in filtered.columns:
    ptier_view = (
        filtered.dropna(subset=["price_tier"])
        .groupby("price_tier", as_index=False)
        .agg(total_revenue=("ASIN Revenue", "sum"), total_sales=("ASIN Sales", "sum"))
        .sort_values("total_revenue", ascending=False)
    )
    ptier_view["price_tier"] = ptier_view["price_tier"].astype(str)

colC, colD = st.columns(2)

with colC:
    st.caption("Revenue by product type")
    if not pt_view.empty:
        st.bar_chart(
            pt_view.set_index("product_type")["total_revenue"],
            height=350,
        )
    else:
        st.write("No data for current filters.")

with colD:
    st.caption("Revenue by price tier")
    if not ptier_view.empty:
        st.bar_chart(
            ptier_view.set_index("price_tier")["total_revenue"],
            height=350,
        )
    else:
        st.write("No data for current filters.")

colC_pie, colD_pie = st.columns(2)
with colC_pie:
    st.caption("Revenue share by product type")
    st.dataframe(pt_view, hide_index=True)
with colD_pie:
    st.caption("Revenue share by price tier")
    st.dataframe(ptier_view, hide_index=True)

# ---------- 7. TOP ASINS TABLE ----------

st.markdown("---")
st.subheader("Top ASINs (current filters)")

top_filtered = (
    filtered.sort_values("ASIN Revenue", ascending=False)
    .loc[
        :,
        [
            "ASIN",
            "Title",
            "brand_clean",
            "product_type",
            "Price",
            "ASIN Sales",
            "ASIN Revenue",
            "Review Count",
            "Reviews Rating",
            "is_large_screen_like",
            "is_rechargeable",
            "is_automotive_targeted",
        ],
    ]
    .head(50)
)

st.dataframe(
    top_filtered,
    use_container_width=True,
    hide_index=True,
)

st.caption("Showing up to 50 ASINs, sorted by revenue. Use the sidebar filters to focus on a specific segment or competitor.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 47.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 26.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.4/208.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━

2025-11-23 20:37:08.199 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-23 20:37:08.200 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 20:37:08.201 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 20:37:08.227 
  command:

    streamlit run /Users/sumiaoc/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-23 20:37:08.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 20:37:08.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 20:37:08.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ig

DeltaGenerator()